In [4]:
import os
import json
import re
from collections import Counter

regex = r"(?i)(?:log(?:ger)?\w*)\s*\.\s*(?:log|error|info|warn|fatal|debug|trace|off|all)\s*\([^;]*\)"

def process_directory(directory):
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isdir(filepath):
            process_directory(filepath)
        elif filename.endswith('.json'):
            process_file(filepath)

def process_file(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
        method_code = data.get('methodCode', '')
        log_variables = data.get('logVariables', [])
        
        for match in re.finditer(regex, method_code):
            logging_statement = match.group(0)
            
            if all(var in logging_statement for var in log_variables):
                start_index = match.start()
                line_count = Counter(method_code[:start_index])['\n']
                start_line = max(0, line_count - 15)
                preceding_lines = method_code.split('\n')[:start_line]
                start_index = len('\n'.join(preceding_lines)) + 1 if preceding_lines else 0
                data['methodCode'] = method_code[start_index:match.end()]
                
                with open(filepath, 'w') as f:
                    json.dump(data, f)
                break
# ...

process_directory('/Users/liyichen/data/')
